In [ ]:
from datasets import load_dataset
from latex_to_image import save_latex_as_image
from modify_latex import modify_numeric_values
from IPython.display import Image, display
import os, datetime

# Load dataset
dataset = load_dataset("staghado/ArXiv-tables")["train"]
os.makedirs("outputs", exist_ok=True)

for i in range(len(dataset)):
    latex = dataset[i]["latex_content"]
    if "\\begin{tabular}" not in latex: 
        continue

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    name_base = f"outputs/sample_{i}_{timestamp}"

    try:
        # Render and display original image
        save_latex_as_image(latex, outname=f"{name_base}_original")
        display(Image(filename=f"{name_base}_original.png"))
    except Exception as e:
        print(f"Table {i+1} failed to render. Skipping. Reason: {e}")
        continue

    # Ask if the user wants to modify the table
    should_modify = input(f"Do you want to modify table {i+1}? (Y/N): ").strip().upper()
    if should_modify != "Y":
        continue

    # Ask how many values to modify
    try:
        n_changes = int(input("How many values would you like to modify?: ").strip())
    except ValueError:
        print("Invalid number. Skipping this table.")
        continue

    modified_latex = latex
    for j in range(n_changes):
        old_val = input(f"Original value #{j+1}: ").strip()
        new_val = input(f"New value #{j+1}: ").strip()
        modified_latex = modify_numeric_values(modified_latex, old_val, new_val)

    try:
        # 📷 Render and display modified image
        save_latex_as_image(modified_latex, outname=f"{name_base}_modified")
        display(Image(filename=f"{name_base}_modified.png"))
        print("Modification complete. Image saved.\n")
    except Exception as e:
        print(f"Failed to render modified table: {e}")